In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import cv2


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
#for dirname, _, filenames in os.walk('/kaggle/input'):
    #for filename in filenames:
        #print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
#Ignore
# GFG :Read the video from specified path
#https://docs.opencv.org/2.4/modules/highgui/doc/reading_and_writing_images_and_video.html?highlight=videocapture%20set#bool%20VideoCapture::set%28int%20propId,%20double%20value%29
#cam = cv2.VideoCapture("kaggle/input/include/Adjectives_6of8/Adjectives/87. hot/MVI_5295.MOV")
#cam.isOpened()
#Doesn't work, retval : false

In [ ]:
#Use following command b4 importing napari:
#pip install napari[all] napari_video
#if needed:
#pip install xgboost 
#Resources- links used
#https://github.com/janclemenslab/napari-video
#https://forum.image.sc/t/working-with-mov-and-other-movie-files-in-python-dask/34053

In [ ]:
import napari
from napari_video.napari_video import VideoReaderNP
# /kaggle/input/include/Adjectives_6of8/Adjectives/87. hot/MVI_5295.MOV
#vr = VideoReaderNP('/kaggle/input/include/Adjectives_6of8/Adjectives/87. hot/MVI_5295.MOV')
#print(vr)

TRY THIS: First, we load the movie to be annotated. Since behavior movies can be quite long, we will use a lazy loading strategy (i.e., we will only load the frames as they are used). Using dask-image, we can construct an object that we can pass to napari for lazy loading in just one line. For more explanation on using dask to lazily load images in napari, see this tutorial.

In [ ]:
vdos = []
vdos.append(VideoReaderNP('/kaggle/input/include/Adjectives_6of8/Adjectives/87. hot/MVI_5136.MOV'))
vdos.append(VideoReaderNP('/kaggle/input/include/Adjectives_6of8/Adjectives/87. hot/MVI_5297.MOV'))
vdos.append(VideoReaderNP('/kaggle/input/include/Adjectives_6of8/Adjectives/89. warm/MVI_9495.MOV'))
vdos.append(VideoReaderNP('/kaggle/input/include/Adjectives_6of8/Adjectives/89. warm/MVI_9336.MOV'))
vdos.append(VideoReaderNP('/kaggle/input/include/Adjectives_6of8/Adjectives/90. cool/MVI_9337.MOV'))
vdos.append(VideoReaderNP('/kaggle/input/include/Adjectives_6of8/Adjectives/90. cool/MVI_9338.MOV'))
#vdos.append(VideoReaderNP('/kaggle/input/include/Adjectives_6of8/Adjectives/91. new/MVI_9261.MOV'))
#vdos.append(VideoReaderNP('/kaggle/input/include/Adjectives_6of8/Adjectives/91. new/MVI_9421.MOV'))
#vdos.append(VideoReaderNP('/kaggle/input/include/Adjectives_6of8/Adjectives/88. cold/MVI_5140.MOV'))

#7-10 index test
vdos.append(VideoReaderNP('/kaggle/input/include/Adjectives_6of8/Adjectives/87. hot/MVI_5295.MOV'))
vdos.append(VideoReaderNP('/kaggle/input/include/Adjectives_6of8/Adjectives/90. cool/MVI_9339.MOV'))
vdos.append(VideoReaderNP('/kaggle/input/include/Adjectives_6of8/Adjectives/89. warm/MVI_5302.MOV'))
#vdos.append(VideoReaderNP('/kaggle/input/include/Adjectives_6of8/Adjectives/91. new/MVI_5148.MOV'))
#vdos.append(VideoReaderNP('/kaggle/input/include/Adjectives_6of8/Adjectives/88. cold/MVI_9490.MOV'))
#/kaggle/input/include/Clothes_2of2/Clothes/46. Clothing/MVI_5181.MOV

In [ ]:
Y_train = pd.DataFrame(columns = ["Class"])
Y_test = pd.DataFrame(["hot","cool","warm"],columns = ["Class"])
labels = ["hot","hot","warm","warm","cool","cool"]
Y_train["Class"] = labels
Y_train, Y_test

In [ ]:
from sklearn.preprocessing import LabelEncoder
le_y = LabelEncoder()
le_y.fit(Y_train)
Y_train = le_y.transform(Y_train["Class"])
Y_test = le_y.transform(Y_test["Class"])
Y_train

In [ ]:
Y_test

In [ ]:
#Ignore this nash, this is for openPose

!git clone https://github.com/Hzzone/pytorch-openpose
%cd pytorch-openpose

#https://www.kaggle.com/code/rkuo2000/openpose-pytorch

In [ ]:
#this box is for openpose, ignore this nash
import matplotlib.pyplot as plt
import copy
import numpy as np
from src import model
from src import util
#https://github.com/Hzzone/pytorch-openpose/blob/master/src/util.py
from src.body import Body
from src.hand import Hand

In [ ]:
!cp /kaggle/input/openpose-pretrained-models/body_pose_model.pth model
!cp /kaggle/input/openpose-pretrained-models/hand_pose_model.pth model
#it's asked to store pretrained models in folder named models

In [ ]:
body_estimation = Body('model/body_pose_model.pth')
hand_estimation = Hand('model/hand_pose_model.pth')
#.pth files are ML models created by PyTorch machine learning framework for Python.

In [ ]:
def get_all_hand_peaks(hands_list,oriImg):
    all_hand_peaks = []
    for x, y, w, is_left in hands_list:
        peaks = hand_estimation(oriImg[y:y+w, x:x+w, :]) #pretrained model, last layer: convolution_param-> num_output: 22, 21 Keypoints
        #peaks is coordinates wrt x,y top left.
        #adjust coordinates of keypoints
        peaks[:, 0] = np.where(peaks[:, 0]==0, peaks[:, 0], peaks[:, 0]+x)
        peaks[:, 1] = np.where(peaks[:, 1]==0, peaks[:, 1], peaks[:, 1]+y) 
        all_hand_peaks.append(peaks)
    return all_hand_peaks


In [ ]:
def generate_hand_keypoints_for_one_video(video):
    vdo_canvas = []
    Num_of_frames = len(video)
    for f in range(Num_of_frames):
        oriImg = video[f]

        candidate, subset = body_estimation(oriImg)
        canvas = copy.deepcopy(oriImg)
        hands_list = util.handDetect(candidate, subset, oriImg)
        all_hand_peaks = get_all_hand_peaks(hands_list,oriImg)
        canvas = util.draw_handpose(canvas, all_hand_peaks)
        vdo_canvas.append(canvas)
    return vdo_canvas

In [ ]:
#test_vdo = '/kaggle/input/include/Adjectives_6of8/Adjectives/87. hot/MVI_5138.MOV' This is labelled in hot but it is cold.
test_image = vdos[0][27]
#Extract a frame
import matplotlib.pyplot as plt
# image = plt.imread(test_image) 
#imread will take filename as a string
plt.figure(figsize=(10,10))
plt.imshow(test_image)
plt.axis('off')
plt.show()

Detect Body

keypoints index : index = int(subset[n][i])

keypoints coord : x,y = candidate[index][0:2]
Candidates each element is a list of 4 elemnts. what are last 2?  first two are x,y co ordinates

In [ ]:
oriImg = vdos[0][27]

candidate, subset = body_estimation(oriImg) #retval: 2 ndarrays
#canvas is ndarray
canvas = copy.deepcopy(oriImg) #recursively copies, so that nested objs are also copied truly and any chnage on copy doesn't affect the original
canvas = util.draw_bodypose(canvas, candidate, subset)
print(len(candidate)) # number of keypoints
print(len(subset))    # number of persons

In [ ]:
#https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/src/openpose/hand/handDetector.cpp

hands_list = util.handDetect(candidate, subset, oriImg)
''' handDetect:
    return value: [[x, y, w, True if left hand else False]].
    width=height since the network require squared input.
    x, y is the coordinate of top left 
'''
all_hand_peaks = get_all_hand_peaks(hands_list,oriImg)
canvas = util.draw_handpose(canvas, all_hand_peaks)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(canvas) #canvas[:, :, [2, 1, 0]]
plt.axis('off')
plt.show()

In [ ]:
#https://github.com/CMU-Perceptual-Computing-Lab/openpose/blob/master/src/openpose/hand/handDetector.cpp
oriImg = vdos[4][23]

candidate, subset = body_estimation(oriImg) #retval: 2 ndarrays
#canvas is ndarray
canvas = copy.deepcopy(oriImg) #recursively copies, so that nested objs are also copied truly and any chnage on copy doesn't affect the original
canvas = util.draw_bodypose(canvas, candidate, subset)
print(len(candidate)) # number of keypoints
print(len(subset))    # number of persons

hands_list = util.handDetect(candidate, subset, oriImg)
''' handDetect:
    return value: [[x, y, w, True if left hand else False]].
    width=height since the network require squared input.
    x, y is the coordinate of top left 
'''
all_hand_peaks = get_all_hand_peaks(hands_list,oriImg)

canvas = util.draw_handpose(canvas, all_hand_peaks)
plt.figure(figsize=(10,10))
plt.imshow(canvas) #canvas[:, :, [2, 1, 0]]
plt.axis('off')
plt.show()

In [ ]:
hands_list

In [ ]:
all_hand_peaks

In [ ]:
print(len(vdos))
len(vdos[0]), vdos[0]

In [ ]:
#one vdo generate hand keypoints
vdo_canvas = generate_hand_keypoints_for_one_video(vdos[0])

In [ ]:
plt.figure(figsize=(10,20))
plt.imshow(vdo_canvas[14]) #canvas[:, :, [2, 1, 0]]
plt.axis('off')
plt.show()

In [ ]:
type(vdos[0]),vdos[6]

In [ ]:
#for all vdo generate hand keypoints
import random

num_of_vdos = len(vdos)
vdos_keypoints = []
vdos_canvas = []
for i in range(num_of_vdos):
    #vdo_canvas = generate_hand_keypoints_for_one_video(vdos[i])
    vdo_canvas = []
    vdo_keypoints = []
    Num_of_frames = len(vdos[i])
    print(i,Num_of_frames)
    frame_index_list =  random.sample([i for i in range(Num_of_frames)],49)
    for f in frame_index_list:
        print(f)
        try:
            oriImg = vdos[0][f]
        except:
            continue
        if(oriImg is None):
            continue
        candidate, subset = body_estimation(oriImg)
        canvas = copy.deepcopy(oriImg)
        hands_list = util.handDetect(candidate, subset, oriImg)
        all_hand_peaks = get_all_hand_peaks(hands_list,oriImg)
        canvas = util.draw_handpose(canvas, all_hand_peaks)
        vdo_keypoints.append(all_hand_peaks)
        vdo_canvas.append(canvas)
    vdos_canvas.append(vdo_canvas)
    vdos_keypoints.append(np.array(vdo_keypoints,dtype = 'float32'))

In [ ]:
#Ignore this box, I was experimenting
#for one vdo, convert frames into np array
'''vdo_frames = vdos[0][0]
n = len(vdo_frames)
vdo_frames_np_arr = []
for i in range(n):
    vdo_frames_np_arr.append(np.array(vdo_frames[i],dtype = 'float32'))
'''

In [ ]:
#useless ish, in our list of vdo each element is of type napari_video.napari_video.VideoReaderNP and the following code is converting it into np.ndarray
#Each vdo has frames, each frame is of type np.ndarray

#for all vdos
'''
vdo_np_arr = [] # list of np array of np array of frames
#np array of frames is a vdo.
for j in range(len(vdos)):
    vdo_frames = vdos[j]
    n = len(vdo_frames)
    vdo_frames_np_arr = []
    for i in range(n):
        vdo_frames_np_arr.append(np.array(vdo_frames[i],dtype = 'float32'))
    vdo_np_arr.append(np.array(vdo_frames_np_arr,dtype = 'float32'))
    '''

In [ ]:
#Ignore this box as well nash, again was trying to understand the object type and dimensions
'''vr = vdos[0][0]
type(vdos[0][0]),type(vdos[0]), type(vdo_np_arr[0])'''

In [ ]:
len(vdos_keypoints)

In [ ]:
#vdos_keypoints

In [ ]:
# f = open("VideoKeypoints.txt","w+")
# for i in range(11):
#     f.write(str(vdos_keypoints[i]))
# f.close()

In [ ]:
#shape=(1, 49, 2, 21, 2), 1 vdo, 49 frames, 2 hands, 21 keypoints of 2 dimensions. 21*2 = 42 for a hand. 2 hands so, 42*2 = 84
X = pd.DataFrame(columns = [i for i in range(84*49)])
X #each row will be a video of 30 frames. 
#pick 49 frames of vdo
col = 0
for vdo in vdos_keypoints:
    for frame in vdo:
        for hands in frame:
            X[i],X[i+1] = map(list,zip(*hands))
            i = i+2
X

In [ ]:
obj_list = X.select_dtypes(include = 'object').columns.to_list()
for col in obj_list:
    X[col] = le_y.fit_transform(X[col])

In [ ]:
X.info()

In [ ]:
#1 vdo, 3 frames, 2 hands, 5 keypoints of 2D
# temp = [[  [ [[0,0],[0,0],[1,1],[0,0],[2,2]] , [[0,0],[0,0],[1,1],[0,0],[2,2]] ]  ,  [ [[0,0],[0,0],[1,1],[0,0],[2,2]] , 
#     [[0,0],[0,0],[1,1],[0,0],[2,2]] ],  [ [[0,0],[0,0],[1,1],[0,0],[2,2]] , [[0,0],[0,0],[1,1],[0,0],[2,2]] ],   ]]

In [ ]:
# import pandas as pd
# X = pd.DataFrame(columns = [i for i in range(12)])
# X #each row will be a video of 30 frames. 
# #pick 49 frames of vdo
# i = 0
# for vdo in temp:
#     for frame in vdo:
#         for hand in frame:
#             X[i],X[i+1] = map(list,zip(*hand))
#             i = i+2
# X

In [ ]:
X_test = X.tail(3)
X_train = X.head(6)

In [ ]:
X_train.shape

In [ ]:
#Caution runnning this takes alot of time, use X_test.shape instead
X_test.shape

In [ ]:
X_train.info()

In [ ]:
Y_train

In [ ]:
from xgboost import XGBClassifier
model = XGBClassifier()

In [ ]:
model.fit(X_train,Y_train)
print(model)

In [ ]:
Y_pred = model.predict(X_test)
Y_pred

In [ ]:
Y_pred_cat = le_y.inverse_transform(Y_pred)
Y_pred_cat

Use images along with the keypoints vectors to classify